In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from PIL import Image, UnidentifiedImageError
import os
from torch.utils.data import Dataset, DataLoader
import requests
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm
from io import BytesIO

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
data = pd.read_excel('/kaggle/input/adobe-behaviour-simulation/behaviour_simulation_train.xlsx')

In [4]:
photo_data = data[data['media'].str.startswith('[Photo')]
video_data = data[data['media'].str.startswith('[Video')]
gif_data = data[data['media'].str.startswith('[Gif')]

In [5]:
train_transform = A.Compose([
    A.Resize(height=224, width=224), #resize 
    A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
             A.RandomBrightnessContrast() 
             ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

In [6]:
li=[]
img_path = '/kaggle/input/adobe-images/data'
for item in os.listdir(img_path):
    po=[]
    full_path = os.path.join(img_path,item)
    if(full_path.endswith('.jpeg')):
        po.append(full_path)
        s = str(item)
        s = s[6:-5]
        s = int(s)
        label = torch.tensor(float(photo_data.iloc[s-1]['likes']))
        img = Image.open(full_path)
        img = np.array(img,dtype=np.float32)/255
        if(img.shape[2]!=3):
            continue
#         augmented = train_transform(image=img)
#         img = augmented['image']
        po.append(s)
        li.append(po)

In [7]:
img = Image.open('/kaggle/input/adobe-images/data/image_1046.jpeg')
img = np.array(img,dtype=np.float32)/255

In [8]:
len(li)

4621

In [21]:
from sklearn.model_selection import train_test_split
train_li, val_li = train_test_split(li, test_size = 0.1, random_state = 42)

In [22]:
class dataset(Dataset):
    def __init__(self, list_, transform, data_):
        super().__init__()
        self.li = list_
        self.transform = transform
        self.data_ = data_
    def __len__(self):
        return len(self.li)
    def __getitem__(self, idx):
        path = self.li[idx][0]
        iddx = self.li[idx][1]
        label = torch.tensor(float(self.data_.iloc[iddx-1]['likes']))
        img = Image.open(path)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        img = np.array(img,dtype=np.float32)/255
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return {
            "image" : img,
            "label" : label,
            "iddx" : iddx
        }

In [23]:
train_data_set = dataset(train_li, train_transform, photo_data)
val_data_set = dataset(val_li, train_transform, photo_data)

In [38]:
train_load = DataLoader(train_data_set, batch_size = 128, shuffle = True, num_workers = 4)
val_load = DataLoader(val_data_set, batch_size = 32, shuffle = True, num_workers = 4)

In [25]:
from torchvision.models import efficientnet_b2

eff = efficientnet_b2(pretrained = True)

num_features = eff.classifier[1].in_features
eff.classifier = nn.Sequential(
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(num_features, 512),
    nn.ReLU(),
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(512, 128),
    nn.ReLU(),
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(128, 1),
)

print(num_features)

for param in eff.parameters():
    param.requires_grad = True

for param in eff.features.parameters():
    param.requires_grad = False

eff = eff.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1408


In [26]:
eff = eff.to(device)

In [27]:
lossfx = nn.MSELoss()
optimizer = torch.optim.Adam(eff.parameters(), lr = 0.0001)
# acc = torchmetrics.Accuracy(task = 'multiclass', num_classes = 2)
# acc = acc.to(device)

In [35]:
# class RMSELoss(nn.Module):
#     def _init_(self):
#         super()._init_()
#         self.ms = nn.MSELoss()
        
#     def forward(self,yhat,y):
#         return torch.sqrt(self.ms(yhat,y))

class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        mse_loss = self.mse(y_pred, y_true)
        rmse_loss = torch.sqrt(mse_loss)
        return rmse_loss

In [40]:
rmse = RMSELoss()
ms = nn.MSELoss()

In [43]:
Epoch = 50
for epoch in tqdm(range(Epoch)):
    net_loss = 0.0
    eff.train()
    for da in tqdm(train_load):
#         x = data['image'].to(device)
#         y = data['label'].to(device)
        x = da['image'].to(device)
        y = da['label'].to(device)
        pred = eff(x)
        loss_ = lossfx(pred.squeeze(), y.type(torch.float))
        net_loss += loss_.item()
        del x
        del y
        optimizer.zero_grad()
        loss_.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1} :- {net_loss}")
    eff.eval()
    net_loss = 0.0
    with torch.inference_mode():
        for da in tqdm(val_load):
            x = da['image'].to(device)
            y = da['label'].to(device)
            pred = eff(x)
            loss_ = ms(pred.squeeze(), y.type(torch.float))
            net_loss += loss_.item()
        net_loss /= len(val_load)
        print(f"Validation Loss - {torch.sqrt(torch.tensor(net_loss)).item()}")

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch 1 :- 703740524.9375


  0%|          | 0/15 [00:00<?, ?it/s]

Validation Loss - 2179.1279296875


...

Epoch 19 :- 705967698.4375


  0%|          | 0/15 [00:00<?, ?it/s]

Validation Loss - 2171.472412109375


  0%|          | 0/33 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [44]:
import pickle

with open('effb2.pkl', 'wb') as f:
    pickle.dump(eff, f)